In [3]:
import xarray as xr
import numpy as np
import dask
from dask.diagnostics import ProgressBar
import matplotlib.pylab as plt
import my_funs

from dask.distributed import Client, LocalCluster

# cluster = LocalCluster()
# client = Client(cluster)
# client = Client(processes=False)

dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/"
out_dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/working/"

percent_cover = (
    xr.open_dataarray(dir + "data/processed_data/percent_cover/percent_cover.nc") * 100
)
percent_cover = percent_cover.loc["1984":"2013"]
percent_cover = percent_cover.round(4)
# If a class is 0 change it no nan to prevent false zeros in diff later
percent_cover = percent_cover.where(percent_cover != 0)

Calculate natural variability for the growing season

In [ ]:

lai_growing = xr.open_dataarray(dir+"data/processed_data/noaa_nc/lai_fapar/resampled/lai_growing_mean.nc")
lai_growing = lai_growing.rename({"latitude":"lat","longitude":"lon"})

lai_growing_trend_ds = xr.open_dataset(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_growing_trend_mean.nc"
)
lai_growing_trend_ds = lai_growing_trend_ds.rename({"latitude":"lat","longitude":"lon"})
lai_growing_trend = lai_growing_trend_ds["trend"]
lai_growing_trend_total = lai_growing_trend.where(
    lai_growing_trend_ds["h"] == 1
)  # where trend is sigificant

# There is a tiny difference between lat and lon of the dataset due to rounding
# We make them equal
percent_cover["lat"] = lai_growing_trend_total["lat"]
percent_cover["lon"] = lai_growing_trend_total["lon"]

t_n = percent_cover.shape[0]
band_n = percent_cover.shape[1]
lat_n = percent_cover.shape[2]
lon_n = percent_cover.shape[3]
n_total = np.isfinite(lai_growing.isel(time=0)).sum()

# Some setting
winsize = 7  # Window size to search for stable pixels around a central pixel
thresh = (
    2  # Threshold to consider a change in ladn cover happended over the past 30 years
)

lai_growing_trend_nv, lai_growing_trend_lcc, changed_pixels = my_funs.estimate_lcc_trend(
    percent_cover, lai_growing_trend_total, thresh, winsize
)
lai_growing_changed = lai_growing.where(changed_pixels)

lai_growing_trend_nv.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_growing_trend_nv.nc"
)
lai_growing_trend_lcc.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_growing_trend_lcc.nc"
)
lai_growing_trend_total.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_growing_trend_total.nc"
)
lai_growing_changed.to_netcdf(
    dir + "data/processed_data/noaa_nc/lai_fapar/trend/lai_growing_changed.nc"
)
changed_pixels.to_netcdf(dir + "data/processed_data/noaa_nc/lai_fapar/trend/changed_pixels.nc")

In [30]:
lai_monthly_trend_ds = xr.open_dataset(dir+"data/processed_data/noaa_nc/lai_fapar/trend/lai_monthly_trend_mean.nc")
lai_monthly_trend_ds = lai_monthly_trend_ds.rename({"latitude":"lat","longitude":"lon"})
percent_cover["lat"] = lai_monthly_trend_ds["lat"]
percent_cover["lon"] = lai_monthly_trend_ds["lon"]

In [39]:
trend_total_lst = []
trend_nv_lst = []
trend_lcc_lst = []

for k in np.arange(5,11):
    tmp = lai_monthly_trend_ds.sel(months=k)
    lai_tmp_trend_total = tmp["trend"]
    # Find the significant slope estimates
    lai_tmp_trend_total = lai_tmp_trend_total.where(
        tmp["h"] == 1
    )
    lai_tmp_trend_nv, lai_tmp_trend_lcc, changed_pixels = my_funs.estimate_lcc_trend(
        percent_cover, lai_tmp_trend_total, thresh, winsize
    )
    trend_total_lst.append(lai_tmp_trend_total)
    trend_nv_lst.append(lai_tmp_trend_nv)
    trend_lcc_lst.append(lai_tmp_trend_lcc)

trend_total_lst_monthly = xr.concat(trend_total_lst,dim="months").assign_coords({"months": np.arange(5, 11)})
trend_nv_lst_monthly = xr.concat(trend_nv_lst,dim="months").assign_coords({"months": np.arange(5, 11)})
trend_lcc_lst_monthly = xr.concat(trend_lcc_lst,dim="months").assign_coords({"months": np.arange(5, 11)})
trend_total_lst_monthly.to_netcdf(dir+"/data/processed_data/noaa_nc/lai_fapar/trend/trend_total_lst_monthly.nc")
trend_nv_lst_monthly.to_netcdf(dir+"/data/processed_data/noaa_nc/lai_fapar/trend/trend_nv_lst_monthly.nc")
trend_lcc_lst_monthly.to_netcdf(dir+"/data/processed_data/noaa_nc/lai_fapar/trend/trend_lcc_lst_monthly.nc")